In [ ]:
!pip install mlxtend
#installing machine learning extensions

In [ ]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

#Importing necessary packages

In [ ]:
from google.colab import files
files.upload()


In [ ]:
df = pd.read_csv("Market_Basket_Optimisation.csv", header=None)

In [ ]:
df.shape

In [ ]:
#Preprocessing
transactions = [] #creating emptylist to append later
for i in range(len(df)):
    row = df.iloc[i].dropna().tolist()  #cleaning and changing to list  as apyrori needs list not df
    transactions.append([str(item).strip() for item in row if str(item).strip() != ''])



In [37]:
print(" Number of transactions:", len(transactions))
print(" Example transaction:", transactions[0])

 Number of transactions: 7501
 Example transaction: ['shrimp', 'almonds', 'avocado', 'vegetables mix', 'green grapes', 'whole weat flour', 'yams', 'cottage cheese', 'energy drink', 'tomato juice', 'low fat yogurt', 'green tea', 'honey', 'salad', 'mineral water', 'salmon', 'antioxydant juice', 'frozen smoothie', 'spinach', 'olive oil']


In [ ]:
!pip install apyori

In [ ]:
from apyori import apriori

#using apriori rule if an itemset is frequent all its subset also is frequent
#there are other rules I can explore like fp growth and eclat;equivalent clustering
#we can tweak the support
rules = apriori(transactions=transactions,
                min_support=0.003,
                min_confidence=0.2,
                min_lift=3,
                min_length=2)

#choosing support confidence and lift and length, min_lenghth is like atleast presentn in 2 baskets .
#lets say a product is bought 3 times 7 days a week /total transactions 7501.This is a 1 week data
#Only in 1 basket
#1 will create no asscociation rule
#Lift >1 is positive association

In [ ]:
#Converting results to list
results = list(rules)

In [ ]:
results

In [ ]:
#transfering list to dataframe. Apyori cannot work on df ,hence after
#rules generated creating a lhs and rhs with Aentecedent and consequent respectively

def susnata_organize(results):
    lhs         = [tuple(result[2][0][0])[0] for result in results]
    rhs         = [tuple(result[2][0][1])[0] for result in results]
    supports    = [result[1] for result in results]
    confidences = [result[2][0][2] for result in results]
    lifts       = [result[2][0][3] for result in results]
    return pd.DataFrame(list(zip(lhs, rhs, supports, confidences, lifts)),
                        columns=['Left Hand Side Antecedent ', 'Right Hand SideConsequent', 'Support', 'Confidence', 'Lift'])


In [ ]:

rules_df = susnata_organize(results)

In [39]:
 rules_df.shape
#80 rules have been created

(80, 5)

In [38]:
# I can customise as per business requirements Lift postive indicate the rule is stronger than random chance
rules_df.head()

,Left Hand Side Antecedent,Right Hand SideConsequent,Support,Confidence,Lift
0,light cream,chicken,0.004533,0.290598,4.843951
1,mushroom cream sauce,escalope,0.005733,0.300699,3.790833
2,pasta,escalope,0.005866,0.372881,4.700812
3,fromage blanc,honey,0.003333,0.245098,5.164271
4,herb & pepper,ground beef,0.015998,0.323450,3.291994


In [ ]:
#My takeaways
#Its like flipkart Amazon swiggy zepto style people who bought
#This also bought this ,Cross-selling bundels to make more revenue

#Rule light cream → chicken
# Interpretation:
# If a customer buys light cream, they are likely to also purchase chicken.
# This suggests that chicken can be recommended alongside light cream as a cross-sell opportunity.
#Lhs may have more than 1 items ,Rhs by default has 1


In [ ]:
# Rule: light cream- chicken
# Support = 0.004533 (0.45%) #although support is low
# Confidence = 0.29 (29% of light cream buyers also buy chicken)
# Lift = 4.8 (4.8x stronger than chance)
#Lift is important as confidence can be high if consequent is already frequent
# Business Action: Place chicken near dairy aisle, or recommend chicken in-app when light cream is added to cart.
#Impulse or receipe driven can further drive buying
